In [37]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import csv
from IPython import display

In [38]:
%%capture
# finds distance covered from index 0 to last index, moving left to right
# doesn't return to original index (full circle), stops at last index
def find_dist(path):       
    # return distance covered float
    return sum(np.sqrt((( path[i - 1][0] - path[i][0] ) ** 2) + (( path[i - 1][1] - path[i][1] ) ** 2)) for i in range(1, len(path)))

# crosses genes from both parents, weighting the probability of either one being chosen for any given gene
# probability controlled by PARENT_PROB_WEIGHT
def create_new_children(parents, mutations, pop_size, PARENT_PROB_WEIGHT): 
    best_child_fitness = 1000000
    second_best_child_fitness = 1000000
    
    best_child = []
    second_best_child = []
    for i in range(pop_size):
        # create child path
        child = [parents[0][0]]

        # iterate through the parents, passing genes to the child, giving weight to the first parent
        # skip the starting point
        for i in range(1, len(parents[0])):
            # chooses a parent (index) based on the weight given
            prob = random.choices([0, 1], weights=(PARENT_PROB_WEIGHT, 1 - PARENT_PROB_WEIGHT))[0]

            # determines the parent that was not chosen, in case the first's genes are already in the child
            opposite_prob = abs(prob - 1)

            # add selected parent's gene to child if it is not present in child already
            if parents[prob][i] not in child:
                child.append(parents[prob][i])

            # add alternate parent's gene to child if it is not present in child and if the selected parent's is present
            elif parents[opposite_prob][i] not in child:
                child.append(parents[opposite_prob][i])

            # if both parents fail to provide a gene the child does not have, find a gene from the selected parent
            # then insert the gene into the child, regardless of what index the parent's gene was found
            else:
                for x in range(1, len(parents[prob]) - 1):
                    if parents[prob][x] not in child:
                        child.append(parents[prob][x])

        for i in range(mutations):
            index_1 = random.randint(1, len(child) - 1)
            index_2 = random.randint(1, len(child) - 1)
            
            child[index_1], child[index_2] = child[index_2], child[index_1] 

        
        child_fitness = find_dist(child)
        if child_fitness < best_child_fitness:
            best_child_fitness = find_dist(child)
            best_child = child
        
        elif child_fitness < second_best_child_fitness:
            second_best_child_fitness = child_fitness
            second_best_child = child
     
    return [best_child, second_best_child]

In [39]:
# sloppily made this a function to iterate easier
def sim(iteration, ITERATIONS, GENERATIONS, MUTATIONS_PER_CHILD, BIRTH_SIZE, PARENT_PROB_WEIGHT, CHANGE_THRESHOLD):
    fitness_record = []
    time_record = []
    original_parents = [COORDINATES[1:], COORDINATES[1:]]

    # shuffle the two parents randomly
    random.shuffle(original_parents[0])
    random.shuffle(original_parents[1])

    # insert the starting position into both parents
    original_parents[0].insert(0, COORDINATES[0])
    original_parents[1].insert(0, COORDINATES[0])
    #original_parents = [[[25, 0], [13, 3], [10, 45], [19, 1], [16, 48], [0, 30], [22, 50], [44, 41], [31, 1], [6, 41], [50, 23], [40, 45], [42, 7], [46, 12], [0, 23], [34, 48], [28, 50], [4, 12], [50, 30], [2, 36], [8, 7], [1, 17], [49, 17], [37, 3], [48, 36]], [[25, 0], [19, 1], [10, 45], [50, 23], [31, 1], [50, 30], [22, 50], [2, 36], [6, 41], [0, 30], [42, 7], [44, 41], [48, 36], [28, 50], [4, 12], [49, 17], [16, 48], [8, 7], [13, 3], [1, 17], [0, 23], [37, 3], [34, 48], [40, 45], [46, 12]]]
    
    # set the fittest children equal to the original parents initially
    fittest_children = original_parents.copy()

    # set the previous generation's parents equal to the original parents initially
    last_parents = original_parents.copy()
    
    # sequence list for animation export
    ANIM_FRAME_SEQ = [original_parents[0]]
    
    # set the highest parent distance to a large number initially
    parent_dist = [100000000, 100000000]

    # set the number of equivalent children equal to 0 initially
    equiv_children = 0
    
    
# run until the "best" distance is found equiv_children times, to ensure the children really are close to the fittest
    fittest_found = False
    gens = 0
    count = 0
    threshold_reached = 0
    while not fittest_found and gens < GENERATIONS:
        # if generations variable is 0.1, never increment, let the loop terminate when fittest_found
        if GENERATIONS != 0.1:
            gens += 1

        # choose the fittest two children, which become the parents of the next generation
        fittest_children = create_new_children(last_parents, MUTATIONS_PER_CHILD, BIRTH_SIZE, PARENT_PROB_WEIGHT)
        
        # find the distance traveled of the fittest child
        child_dist = [find_dist(fittest_children[0]), find_dist(fittest_children[1])]
        
        
        fitness_record.append([child_dist[0], gens])
        time_record.append([child_dist[0], (time.time() - start_time)])
        # if the child is fitter than the parent, replace the parent with the child
        for i in range(len(child_dist)):
            for x in range(len(parent_dist)):
                # if the first child is fitter than the parent, replace the parent
                
                if child_dist[i] < parent_dist[x]:
                    equiv_children = 0 # reset number of equivalent children
                    count = 0
                    parent_dist[x] = child_dist[i] # save the fitness score
                    last_parents[x] = fittest_children[i] # save the child
                    # add the frame to the list of frames
                    ANIM_FRAME_SEQ.append(fittest_children[i])
                    break

                # if the child is equivalent to both parents, determine the best child
            
            #if child_dist[i] == parent_dist[0] and child_dist[i] == parent_dist[1]: 
            if child_dist[i] == parent_dist[0]:
                equiv_children += 1
                count += 1
                if equiv_children >= NUM_FITTEST_CHILDREN:
                    fittest_found = True
                
                elif count >= NUM_FITTEST_CHILDREN / CHANGE_THRESHOLD:
                    BIRTH_SIZE = round(BIRTH_SIZE * 1.25)
                    count = 0
                    threshold_reached += 1

        # output the progress
            display.clear_output(wait=True)
            display.display(f'Current Best Fitness: {parent_dist[0]} // {equiv_children}/{NUM_FITTEST_CHILDREN}' +
            f' equivalents found // {gens}/{GENERATIONS} generations complete //' +
            f'{iteration}/{ITERATIONS} iterations complete // {threshold_reached} times threshold was reached')

            # determine fittest child if the number of equivalent children reaches the provided number
            
            
    end_time = time.time() # record  final time of end of execution
    
    
    
    return last_parents, ANIM_FRAME_SEQ, original_parents, fitness_record, time_record

In [40]:
#ESSENTIAL VARIABLES
# create a set of the coordinates in tuple form of the set of points
#COORDINATES = [(1,0), (2,0), (3,1), (3,2), (2,3), (1,3), (0,2), (0,1)] # octagon
#COORDINATES = [(0,1), (1,0), (2,0), (3,1), (1.5,2)] # pentagon
#COORDINATES = [(1,10), (3, 8),(0,2), (1,1), (2,0), (3,0), (4,1), (5,2), (4,3), (3,4), (2,4), (1,3), (2,5), (6,7), (8,12), (4,4), (9,9), (4,6), (12,5), (1,6), (2, 7), (8,2), (9,4), (7,6)] # random
#COORDINATES = [(0,2), (1,1), (2,0), (3,0), (4,1), (5,2), (4,3), (3,4), (2,4), (1,3)] # decigon

with open('coordinates.csv', newline='') as f:
    reader = csv.reader(f)
    COORDINATES = list(reader)

for point in COORDINATES:
    point[0] = int(point[0])
    point[1] = int(point[1])

# number of children a parent births per run
# usually use 500
BIRTH_SIZE = 500

# number of children required to have the fittest distance before the final distance is determined
# higher = more accurate, but takes much more time, especially past 200
# usually use 100
NUM_FITTEST_CHILDREN = 100

"""
max. number of mutations per child.
seems as though the higher the number of coordinates gets, the mutation number has a greater effect.
however, too high, and computation time takes too long.
too low, it's less accurate.
3 seems to be the sweet spot
at lower numbers, the mutations per child don't seem to matter
# usually use 1 for small, 3 for large
"""
MUTATIONS_PER_CHILD = 3

# weighted preference for the algorithm to choose genes from the first "more fit" parent
# set to 0.5 to give equal weight, increase to increase weight to first parent, decrease to decrease weight
PARENT_PROB_WEIGHT = 0.6

# generations per iteration
# set to 0.1 to uncap the generation limit; loop will run forever until fittest isfound
GENERATIONS = 0.1

# number of times to repeat the algorithm
ITERATIONS = 1

# num of stagnant children before reducing mutations and increasing birth size
# activates when num stagnant >= equivalent children / this
CHANGE_THRESHOLD = 1

In [41]:
# future should add parameters to change the 6 big variables while iterating
# might be useful in pursuing the "good" eggs and discarding the bad ones
best = 1000000
start_time = time.time()

for i in range(0, ITERATIONS):
 
    last_parents_TEMP, ANIM_FRAME_SEQ_TEMP, original_parents_TEMP, fitness_record_TEMP, time_record_TEMP = sim(i,
                                                ITERATIONS,
                                                GENERATIONS,
                                                MUTATIONS_PER_CHILD,
                                                BIRTH_SIZE,
                                                PARENT_PROB_WEIGHT,
                                                CHANGE_THRESHOLD)
    
    if find_dist(last_parents_TEMP[0]) < best:
        last_parents = last_parents_TEMP
        ANIM_FRAME_SEQ = ANIM_FRAME_SEQ_TEMP
        best = find_dist(last_parents[0])
        original_parents = original_parents_TEMP
        fitness_record = fitness_record_TEMP
        time_record = time_record_TEMP

time_diff = time.time() - start_time

KeyboardInterrupt: 

In [ ]:
# OUTPUT FINAL DATA
# information about old sequence of coordinates, then GA's sequence
print("Start Point:" + str(original_parents[0][0]) + "\n")
print("GA's Best Sequence" + str(last_parents[0]))
print("GA's Best Distance: " + str(find_dist(last_parents[0])) + "\n")

print("Original Sequence:" + str(original_parents[0]))
print("Original Distance: " + str(find_dist(original_parents[0])) + "\n")

pct_change = ( find_dist(original_parents[0]) - find_dist(last_parents[0]) )/ find_dist(original_parents[0]) * 100
print("Percent Improvement:" + str(pct_change) + "%\n")

print("Average Execution Time Per Fresh Start: " + str(time_diff / ITERATIONS) + " seconds")

x, y = np.array(original_parents[0]).T
x_path, y_path = np.array(last_parents[0]).T

plt.title("Original Path")
plt.plot(x, y, "bd")
plt.plot(x, y, "r")
plt.grid()
plt.show()

plt.title("GA's Path")
plt.plot(x_path, y_path, "bd")
plt.plot(x_path, y_path, "g")
plt.grid()
plt.show()

x, y = np.array(fitness_record).T
plt.title("Best Fitness Per Generation")
plt.xlabel("Generation #")
plt.ylabel("Fitness")
plt.plot(fitness_record)
plt.yscale('log')
plt.axhline(151.00638509118642)
plt.grid()
plt.show()
plt.clf()

x, y = np.array(time_record).T
plt.title("Fitness vs. Time")
plt.xlabel("Time Elapsed")
plt.ylabel("Fitness")
plt.plot(y, x, 'b')
plt.axhline(151.00638509118642)
plt.grid()
plt.show()

In [ ]:
"""%%capture
for i in range(len(ANIM_FRAME_SEQ) ):
    plt.title(f'Frame {ANIM_FRAME_SEQ[i]}')
    x_frame, y_frame = np.array(ANIM_FRAME_SEQ[i]).T
    plt.plot(x_frame, y_frame, "bd")
    plt.plot(x_frame, y_frame, "g")
    plt.savefig(f'Frame{len(ANIM_FRAME_SEQ) - (i+1)}')
    plt.grid()
    plt.show()"""